In [ ]:
import hashlib

class Shingling:
    def __init__(self, k):
        self.k = k
        self.shingles = set()

    def construct_shingles(self, document):
        for i in range(len(document) - self.k + 1):
            shingle = document[i:i+self.k]
            self.shingles.add(shingle)

    def compute_hashes(self):
        hashed_shingles = {}
        for shingle in self.shingles:
            hash_value = int(hashlib.sha256(shingle.encode('utf-8')).hexdigest(), 16)
            hashed_shingles[shingle] = hash_value
        return hashed_shingles

# Example Usage
document = "Lorem ipsum dolor sit amet, consectetur adipiscing elit."
k = 5  # Example k value, you can choose any value for k

shingling_obj = Shingling(k)
shingling_obj.construct_shingles(document)
hashed_shingles = shingling_obj.compute_hashes()
print(hashed_shingles)

ID2222: Lab 2 - 